In [1]:
# !pip install statsmodels

In [2]:
from pathlib import Path
import pandas as pd

config_dirs = open("prefixo_dados.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip()) / "Projeto_PA"

pasta = DIR_PREFIXO / "Projeto_PA_validado" / "NMF"
padrao_nome = "NMF_approach_medidas_grupo_{}.csv"

def le_grupos(pasta, padrao_nome):
    grupos = []
    for numero_grupo in range(1, 20):
        nome = padrao_nome.format(numero_grupo) 
        if (pasta / nome).exists():
            grupo = pd.read_csv(pasta / nome, sep=";",encoding="latin1")
            if "Unnamed: 0" in grupo.columns:
                grupo.drop("Unnamed: 0", axis=1)
            if "numero_identificacao" in grupo.columns:
                grupo.set_index("numero_identificacao")
            grupos.append(grupo)
    return grupos
            
        
grupos = le_grupos(pasta, padrao_nome)
        


In [3]:
grupos

[       Unnamed: 0  numero_identificacao data_nascimento  genero  idade   peso  \
 0               0                     4      1923-10-05       1   94.0   85.0   
 1               1                     5      1948-10-07       1   70.0   80.0   
 2               2                     6      1999-03-23       1   19.0  100.0   
 3               3                     7      1978-01-26       0   40.0   65.0   
 4               4                     8      1990-03-07       1   28.0  120.0   
 ...           ...                   ...             ...     ...    ...    ...   
 16262       19256                 21782      1979-05-23       0   41.0   56.0   
 16263       19257                 21783      1969-05-24       1   51.0   93.0   
 16264       19259                 21787      1970-10-19       1   49.0   90.0   
 16265       19260                 21788      1985-03-15       0   35.0   93.0   
 16266       19262                 21793      1971-01-14       1   49.0   99.0   
 
       altura 

In [4]:
path_mapas = pasta / "todos_pacientes" / "NMF_todos_pacientes_categorizados.csv"

mapas = pd.read_csv(path_mapas, sep=";", encoding="latin1")
mapas.set_index("numero_identificacao", inplace=True)
#mapas
#cat_genero = mapas["genero"].apply(lambda genero: 0 if genero ==  "F" else 1)
# cat_genero.name = "cat. genero"
#cat_genero

mapa_variaveis = pd.concat( [mapas["Grupo"], mapas["cat. idade"], mapas["cat. imc"], mapas["genero"], mapas["Whitecoat"]], axis = 1)
mapa_variaveis.dropna(inplace=True)
mapa_variaveis["cat. imc"].isna().sum()

0

In [5]:
mapa_variaveis[['cat. idade', 'cat. imc', 'genero', 'Whitecoat']]

,cat. idade,cat. imc,genero,Whitecoat
numero_identificacao,,,,
5,3,2.0,1,0
6,1,2.0,1,1
7,1,1.0,0,1
8,1,3.0,1,0
9,1,2.0,1,1
...,...,...,...,...
21380,3,3.0,1,0
21400,1,1.0,1,0
21415,3,2.0,0,0


In [6]:
import statsmodels.api as sm
import numpy as np

#X = np.asarray(grupos[0]["Dipping"]).astype(int)[:, np.newaxis]
y_dependente = mapa_variaveis[['Whitecoat']] #, 'cat. imc', 'genero']]
#print (y_dependente)
X_independente = mapa_variaveis[['Grupo', 'genero']]
print (X_independente)
sm_model = sm.MNLogit(y_dependente, X_independente).fit()

# print(sm_model.pvalues)
#dir(sm_model)
sm_model.summary()

                      Grupo  genero
numero_identificacao               
5                         1       1
6                         1       1
7                         1       0
8                         1       1
9                         1       1
...                     ...     ...
21380                     7       1
21400                     7       1
21415                     7       0
21699                     7       0
21720                     7       1

[19144 rows x 2 columns]
Optimization terminated successfully.
         Current function value: 0.632702
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:              Whitecoat   No. Observations:                19144
Model:                        MNLogit   Df Residuals:                    19142
Method:                           MLE   Df Model:                            1
Date:                Sun, 20 Feb 2022   Pseudo R-squ.:                -0.01639
Time:                        12:45:48   Log-Likelihood:                -12112.
converged:                       True   LL-Null:                       -11917.
Covariance Type:            nonrobust   LLR p-value:                     1.000
===============================================================================
Whitecoat=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Grupo           0.3880      0.013     28.900      0.000       0.362       0.414
genero          0.2928      0.028     10.313      0.000       0.237       0.348
===============================================================================
"""

In [7]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])

clf.predict_proba(X[:2, :])


clf.score(X, y)

NameError: name 'X' is not defined

In [ ]:
from sklearn import datasets, linear_model
from sklearn.feature_selection import chi2
print(list(grupos[1].columns))

In [ ]:
import statsmodels.api as sm
import numpy as np

X = np.asarray(grupos[0]["Whitecoat"]).astype(int)[:, np.newaxis]
sm_model = sm.Logit(grupos[0]["genero"], sm.add_constant(X)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

In [ ]:
#    Cálculo do Desvio-padrão Conjunto pelo método D de Cohen
#
#  Sob a hipótese de igualdade de variâncias na população, supõe-se que a variância coletiva (conjunta)
#  da amostra proporciona maior precisão na estimativa da variância populacional do que as variâncias
#  individuais da amostra. Esta precisão mais elevada pode conduzir a um aumento do poder estatístico
#  quando utilizado em testes de hipótese estatística, tais como o teste t de Student.
#  É também empregado no cálculo das medidas de tamanho do efeito.
#  s: desvio-padrão
#  n: tamanho da amostra
#  k: número de tamanhos de amostras
#
from statistics import mean, stdev
from math import sqrt

# test conditions
c0 = [2, 4, 7, 3, 7, 35, 8, 9]
c1 = [i * 2 for i in c0]

cohens_d = (mean(c0) - mean(c1)) / (sqrt((stdev(c0) ** 2 + stdev(c1) ** 2) / 2))

print(cohens_d)

In [ ]:
#print(list(mapas.columns))
mapas[mapas["Grupo"] == 2]["sist 8:30"]
#print(list(mapas.columns))